<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [13]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [14]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [15]:
# Write your code here
summary = pd.DataFrame({
    'Data Type': df.dtypes,
    'Non-Null Count': df.count(),
    'Null Count': df.isnull().sum(),
    'Null Percentage': (df.isnull().mean() * 100).round(2)
})
print("Dataset Summary:")
print(summary.sort_values('Null Percentage', ascending=False))

Dataset Summary:
                           Data Type  Non-Null Count  Null Count  \
AINextMuch less integrated    object            1148       64289   
AINextLess integrated         object            2355       63082   
AINextNo change               object           12498       52939   
AINextMuch more integrated    object           13438       51999   
EmbeddedAdmired               object           16733       48704   
...                              ...             ...         ...   
MainBranch                    object           65437           0   
Age                           object           65437           0   
Employment                    object           65437           0   
Check                         object           65437           0   
ResponseId                     int64           65437           0   

                            Null Percentage  
AINextMuch less integrated            98.25  
AINextLess integrated                 96.40  
AINextNo change             

<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [16]:
# 1. Select numerical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

# 2. Safe calculation functions
def safe_skew(x):
    with np.errstate(all='ignore'):
        return x.skew()

def safe_kurt(x):  # Defined as safe_kurt
    with np.errstate(all='ignore'):
        return x.kurtosis()

# 3. Generate statistics
stats = df[num_cols].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]).T
stats['skewness'] = df[num_cols].apply(safe_skew)
stats['kurtosis'] = df[num_cols].apply(safe_kurt)  # Corrected to use safe_kurt

# 4. Add additional metrics
stats['IQR'] = stats['75%'] - stats['25%']
stats['range'] = stats['max'] - stats['min']
stats['missing'] = df[num_cols].isnull().sum()
stats['missing_pct'] = (df[num_cols].isnull().mean() * 100).round(2)

# Format output
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
print("Numerical Columns Statistics:")
print(stats[['count', 'mean', 'std', 'min', '5%', '50%', '95%', 'max', 
             'IQR', 'range', 'skewness', 'kurtosis', 'missing', 'missing_pct']])

Numerical Columns Statistics:
                        count  \
ResponseId          65,437.00   
CompTotal           33,740.00   
WorkExp             29,658.00   
JobSatPoints_1      29,324.00   
JobSatPoints_4      29,393.00   
JobSatPoints_5      29,411.00   
JobSatPoints_6      29,450.00   
JobSatPoints_7      29,448.00   
JobSatPoints_8      29,456.00   
JobSatPoints_9      29,456.00   
JobSatPoints_10     29,450.00   
JobSatPoints_11     29,445.00   
ConvertedCompYearly 23,435.00   
JobSat              29,126.00   

                                                                  mean  \
ResponseId                                                   32,719.00   
CompTotal           29,638,411,381,149,976,434,844,996,221,255,135,...   
WorkExp                                                          11.47   
JobSatPoints_1                                                   18.58   
JobSatPoints_4                                                    7.52   
JobSatPoints_5                

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [7]:
#Analyse 'Country' column
country_counts = df['Country'].value_counts(dropna=False)

print("Country Value Counts:")
print(country_counts.head(20)) #top 20 most frequent entries
#Identify potential inconsistencies
print("\n Potential Issues:")
#Misex entries
mixed_case = df["Country"].str.islower().sum()>0
print(f"- Mixed case entries:{'Yes' if mixed_case else 'No'}")

#Irrelevant/placeholder entries
irrelevant = df['Country'].str.contains('N/A|unknown|select|test', case=False, na=False).sum()
print(f"- Irrelevant placeholders:{irrelevant} entries")
valid_countries = ['USA','Canada','UK','India','China','Pakistan']
unexpected = ~df['Country'].isin(valid_countries) & df['Country'].notna()
print(f"- Unexpected values: {unexpected.sum()} entries")
print("Sample unexpected values:", df.loc[unexpected, 'Country'].unique()[:5])

Country Value Counts:
Country
United States of America                                11095
NaN                                                      6507
Germany                                                  4947
India                                                    4231
United Kingdom of Great Britain and Northern Ireland     3224
Ukraine                                                  2672
France                                                   2110
Canada                                                   2104
Poland                                                   1534
Netherlands                                              1449
Brazil                                                   1375
Italy                                                    1341
Australia                                                1260
Spain                                                    1123
Sweden                                                   1016
Russian Federation                      

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [8]:
country_map = {
    # Case variations
    'usa': 'USA',
    'Usa': 'USA',
    'u.s.a': 'USA',
    'america': 'USA',
    
    # Common abbreviations
    'uk': 'UK',
    'u.k.': 'UK',
    'united kingdom': 'UK',
    
    # Misspellings
    'canada': 'Canada',
    'cananda': 'Canada',
    
    # Custom rules
    'unknown': None,
    'n/a': None
}

df['Country'] = (df['Country'].str.strip().str.title().replace(country_map).where(df['Country'].notna()))

In [9]:
edlevel_map = {
    'bachelors': 'Bachelor’s degree',
    'bs': 'Bachelor’s degree',
    'ba': 'Bachelor’s degree',
    'masters': 'Master’s degree',
    'ms': 'Master’s degree',
    'phd': 'Doctoral degree',
    'doctoral': 'Doctoral degree',
    'high school': 'Secondary school',
    'some college': 'Some college',
    'college': 'Some college'
}

df['EdLevel'] = (df['EdLevel'].str.strip().str.title().replace(edlevel_map))

In [10]:
def standardize_column(series, mapping):
    return(series.str.strip().str.title().replace(mapping).where(series.notna()))

df['Country'] = standardize_column(df['Country'], country_map)
df['EdLevel'] = standardize_column(df['EdLevel'], edlevel_map)

In [11]:
print("\nStandardized Country Values:")
print(df['Country'].value_counts(dropna=False).head(10))

print("\nStandardized Education Levels:")
print(df['EdLevel'].value_counts(dropna=False).head(10))


Standardized Country Values:
Country
United States Of America                                11095
NaN                                                      6507
Germany                                                  4947
India                                                    4231
United Kingdom Of Great Britain And Northern Ireland     3224
Ukraine                                                  2672
France                                                   2110
Canada                                                   2104
Poland                                                   1534
Netherlands                                              1449
Name: count, dtype: int64

Standardized Education Levels:
EdLevel
Bachelor’S Degree (B.A., B.S., B.Eng., Etc.)                                          24942
Master’S Degree (M.A., M.S., M.Eng., Mba, Etc.)                                       15557
Some College/University Study Without Earning A Degree                                 7651


In [17]:
#Flag remaining non-standard values
valid_countries = ['USA',
    'UK',
    'Canada',
    'India',
    'China',
    'Pakistan',
    'Germany',
    'France',
    'Japan',
    'Australia',
    'Brazil',
    'South Africa',
    'Nigeria',
    'Russia',
    'South Korea',
    'Singapore',
    'Netherlands',
    'Spain',
    'Italy',
    'Mexico']
df['Country_IsStandard'] = df['Country'].isin(valid_countries)
print(f"\nNon-standard countries: {len(df) - df['Country_IsStandard'].sum()}")
print("Sample non-standard:", 
      df.loc[~df['Country_IsStandard'], 'Country'].dropna().unique()[:5])


Non-standard countries: 43000
Sample non-standard: ['United States of America'
 'United Kingdom of Great Britain and Northern Ireland' 'Norway'
 'Uzbekistan' 'Serbia']


### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [21]:
#Split mulitple employment status
employment_split = df['Employment'].str.split(';')
#get all unique employment categories
all_categories = set()
for sublist in employment_split.dropna():
    all_categories.update(sublist)
unique_categories = sorted(all_categories)\
#Create one-hot encoded columns
for category in unique_categories:
    #Clean category name for column naming
    col_name = f"emp_{category.lower().replace(',','').replace(' ','_')}"
    df[col_name] = df['Employment'].apply(
        lambda x: 1 if pd.notna(x) and category in x.split(';') else 0
    )
# Clean up column names
df.columns = df.columns.str.replace(',','')
#Display results
print(df[['Employment'] + [col for col in df.columns if col.startswith('emp_')]])

                Employment  emp_employed_full-time  emp_employed_part-time  \
0      Employed, full-time                       1                       0   
1      Employed, full-time                       1                       0   
2      Employed, full-time                       1                       0   
3       Student, full-time                       0                       0   
4       Student, full-time                       0                       0   
...                    ...                     ...                     ...   
65432  Employed, full-time                       1                       0   
65433  Employed, full-time                       1                       0   
65434  Employed, full-time                       1                       0   
65435  Employed, full-time                       1                       0   
65436   Student, full-time                       0                       0   

       emp_i_prefer_not_to_say  \
0                            

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [23]:
# Calculate missing values and sort
missing_stats = (
    df.isnull().sum()
    .sort_values(ascending=False)
    .to_frame(name='Missing_Count')
    .assign(Missing_Percentage=lambda x: (x['Missing_Count']/ len(df)*100))
)

print("Columns with Highest Missing Values:")
print(missing_stats[missing_stats['Missing_Count'] > 0].head(10))


Columns with Highest Missing Values:
                               Missing_Count  Missing_Percentage
AINextMuch less integrated             64289               98.25
AINextLess integrated                  63082               96.40
AINextNo change                        52939               80.90
AINextMuch more integrated             51999               79.46
EmbeddedAdmired                        48704               74.43
EmbeddedWantToWorkWith                 47837               73.10
EmbeddedHaveWorkedWith                 43223               66.05
ConvertedCompYearly                    42002               64.19
AIToolNot interested in Using          41023               62.69
AINextMore integrated                  41009               62.67


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [25]:
#Select numerical columns
num_cols = df.select_dtypes(include=['int64','float64']).columns

#Choose stratergy - Mean or Median
STRATERGY = 'median'

for col in num_cols:
    if df[col].isnull().sum() > 0: #Only process columns with missing values
        if STRATERGY == 'median':
            fill_value = df[col].median()
        else:
            fill_value = df[col].mean()
        df[col] = df[col].fillna(fill_value)
        print(f"Imputed {df[col].isnull().sum()} missing values in {col} with {STRATERGY} value: {fill_value:,.2f}")

Imputed 0 missing values in WorkExp with median value: 9.00
Imputed 0 missing values in JobSatPoints_1 with median value: 10.00
Imputed 0 missing values in JobSatPoints_4 with median value: 0.00
Imputed 0 missing values in JobSatPoints_5 with median value: 0.00
Imputed 0 missing values in JobSatPoints_6 with median value: 20.00
Imputed 0 missing values in JobSatPoints_7 with median value: 15.00
Imputed 0 missing values in JobSatPoints_8 with median value: 10.00
Imputed 0 missing values in JobSatPoints_9 with median value: 5.00
Imputed 0 missing values in JobSatPoints_10 with median value: 0.00
Imputed 0 missing values in JobSatPoints_11 with median value: 0.00
Imputed 0 missing values in ConvertedCompYearly with median value: 65,000.00
Imputed 0 missing values in JobSat with median value: 7.00


<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [28]:
#select categorail columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    if df[col].isnull().sum() > 0:
        #get most frequent values (mode)
        mode_values = df[col].mode()[0]
        #impute missing values
        df[col] = df[col].fillna(mode_values)

        print(f"Imupted {df[col].isnull().sum()} missing values in '{col}' with mode: '{mode_values}'")

Imupted 0 missing values in 'RemoteWork' with mode: 'Hybrid (some remote, some in-person)'
Imupted 0 missing values in 'CodingActivities' with mode: 'Hobby'
Imupted 0 missing values in 'EdLevel' with mode: 'Bachelor’s degree (B.A., B.S., B.Eng., etc.)'
Imupted 0 missing values in 'LearnCode' with mode: 'Other online resources (e.g., videos, blogs, forum, online community)'
Imupted 0 missing values in 'LearnCodeOnline' with mode: 'Technical documentation;Blogs;Written Tutorials;Stack Overflow'
Imupted 0 missing values in 'TechDoc' with mode: 'API document(s) and/or SDK document(s);User guides or README files found in the source repository;Traditional public search engine'
Imupted 0 missing values in 'YearsCode' with mode: '10'
Imupted 0 missing values in 'YearsCodePro' with mode: '2'
Imupted 0 missing values in 'DevType' with mode: 'Developer, full-stack'
Imupted 0 missing values in 'OrgSize' with mode: '20 to 99 employees'
Imupted 0 missing values in 'PurchaseInfluence' with mode: 'I h

### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [37]:
#Calculate min and max
comp_min = df['ConvertedCompYearly'].min()
comp_max = df['ConvertedCompYearly'].max()

#Normalize in one operation
df_normalized = df.assign(
    ConvertedCompYearly_Normalized = lambda x: (x['ConvertedCompYearly'] - comp_min) / (comp_max - comp_min)
)

<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [53]:
from scipy.stats import skew
# 1. Handle missing values
comp_median = df['ConvertedCompYearly'].median()
comp_clean = df['ConvertedCompYearly'].fillna(comp_median)

# 2. Calculate offset for non-positive values
min_comp = float(comp_clean.min())  # Convert to native Python float
offset = 1 - min_comp if min_comp <= 0 else 0
if offset > 0:
    print(f"Added offset: {offset:.2f}")

# 3. Perform log transformation
log_values = np.log(comp_clean + offset)

# 4. Create new DataFrame (memory-efficient)
df = df.assign(
    ConvertedCompYearly_Log=log_values
)

# 5. Verify results (using native Python floats for formatting)
log_min = float(log_values.min())
log_max = float(log_values.max())
orig_min = float(comp_clean.min())
orig_max = float(comp_clean.max())

print("\nTransformation Summary:")
print(f"Original range: [{orig_min:,.2f}, {orig_max:,.2f}]")
print(f"Log range: [{log_min:.2f}, {log_max:.2f}]")
print(f"Skewness reduced from {skew(comp_clean):.2f} to {skew(log_values):.2f}")


Transformation Summary:
Original range: [1.00, 16,256,603.00]
Log range: [0.00, 16.60]
Skewness reduced from 87.71 to -4.38


### 7. Feature Engineering


In [ ]:
7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:

In [ ]:
#convert to numeric
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')
#create experiencelevel categories
conditions = [
    (df['YearsCodePro'].isna()),
    (df['YearsCodePro'] <= 2),
    (df['YearsCodePro'] <= 5),
    (df['YearsCodePro'] <= 10),
    (df['YearsCodePro'] > 10),
]

choices = [
    'Unknown',
    'Entry-level (0-2 years)',
    'Mid-level (3-5 years)',
    'Senior (6-10 years)',
    'Expert (10+ years)'
]

df['ExperienceLevel'] = np.select(conditions, choices, default='Unknown')



### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
